In [ ]:
!pip install -q fuzzywuzzy python-Levenshtein Levenshtein nltk pandas pyarrow seaborn

In [ ]:
import pandas as pd
import seaborn as sns
from Levenshtein import distance
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk import edit_distance
import os

def compute_Levensthein_sim(str1,str2):
    edit_dist  = edit_distance(str1,str2)
    len_total = max(len(str1),len(str2))
    return ((len_total-edit_dist)/len_total)*100

def match_part(x,index):
    return x.str.split("-----")[0][index-1] if len(x.str.split("-----")[0])>=index  else None



In [ ]:
PATH_TO_FILE="../../resultats_stage/"
FILENAME_INPUT_RESULTS="Dares_1Kaccords_dataset_txt_label_result.parquet"

In [ ]:
df_results=pd.read_parquet(os.path.join(PATH_TO_FILE,FILENAME_INPUT_RESULTS))

In [ ]:
df_results["titre_label"]=df_results[['label']].apply(lambda x : match_part(x,1),axis=1)
df_results["corps_label"]=df_results[['label']].apply(lambda x : match_part(x,2) ,axis=1)
df_results["annexe_label"]=df_results[['label']].apply(lambda x : match_part(x,3),axis=1)
df_results["titre_result_1a"]=df_results[['result_1a']].apply(lambda x : match_part(x,1),axis=1)
df_results["corps_result_1a"]=df_results[['result_1a']].apply(lambda x : match_part(x,2) ,axis=1)
df_results["titre_result_1b"]=df_results[['result_1b']].apply(lambda x : match_part(x,1),axis=1)
df_results["corps_result_1b"]=df_results[['result_1b']].apply(lambda x : match_part(x,2) ,axis=1)
df_results["titre_1a_Levensthein"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :distance(*x),axis=1)
df_results["titre_1a_Levensthein_sim"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :compute_Levensthein_sim(*x),axis=1)
df_results["titre_1a_fuzz_ratio"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :fuzz.ratio(*x),axis=1)
df_results["titre_1a_fuzz_partial_ratio"]=df_results[["titre_label","titre_result_1a"]].apply(lambda x :fuzz.partial_ratio(*x),axis=1)
df_results["titre_1b_Levensthein"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :distance(*x),axis=1)
df_results["titre_1b_Levensthein_sim"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :compute_Levensthein_sim(*x),axis=1)
df_results["titre_1b_fuzz_ratio"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :fuzz.ratio(*x),axis=1)
df_results["titre_1b_fuzz_partial_ratio"]=df_results[["titre_label","titre_result_1b"]].apply(lambda x :fuzz.partial_ratio(*x),axis=1)

In [ ]:
sns.displot(df_results[["titre_1a_fuzz_partial_ratio","titre_1a_fuzz_ratio","titre_1a_Levensthein_sim"]],multiple="dodge",kde=True,bins=10)

In [ ]:
sns.displot(df_results[["titre_1a_fuzz_partial_ratio","titre_1b_fuzz_partial_ratio"]],multiple="dodge",kde=True,bins=10)

In [ ]:
sns.displot(df_results[["titre_1a_fuzz_ratio","titre_1b_fuzz_ratio"]],multiple="dodge",kde=True,bins=10)

In [ ]:
sns.displot(df_results[["titre_1a_Levensthein_sim","titre_1b_Levensthein_sim"]],multiple="dodge",kde=True,bins=10)

In [ ]:
df_results[df_results["titre_1a_Levensthein_sim"].between(80,100)][["titre_label","titre_result_1a","result_1a","titre_1a_Levensthein_sim","titre_1a_fuzz_ratio","titre_result_1b","result_1b","titre_1b_Levensthein_sim"]]